<a href="https://colab.research.google.com/github/kacper-dobrow/tensorflow-course/blob/main/08_intro_to_NLP_in__TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (e.g.: sequences of text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get helper functions

In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-11-05 15:37:38--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-11-05 15:37:38 (105 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we'll be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not)

See the original source here:

https://www.kaggle.com/competitions/nlp-getting-started


In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-11-05 15:37:41--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.12.27, 2607:f8b0:4025:815::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-11-05 15:37:41 (112 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize the data

Reading and writing files in Python:
https://realpython.com/read-write-files-python/

BUT you can use Pandas

Load and preprocess data in tf: https://www.tensorflow.org/tutorials/load_data/csv?hl=pl

In [6]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class - balanced or not
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In case the data was more imbalanced:

https://www.tensorflow.org/tutorials/structured_data/imbalanced_data?hl=pl

In [10]:
# how many total smaples are there
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Visualize random training samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}","(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Can you save
Can you save my
Can you save my heavydirtysoul?

---

Target: 0 (not real disaster)
Text:
@ThatPersianGuy @YOUNGSAFE ?? Eden Hazard as Harden is spot on flopping is identical

---

Target: 1 (real disaster)
Text:
the mv should just be them strutting like they mean it while buildings are burning up in the bg and flames everywhere how cool would that be

---

Target: 1 (real disaster)
Text:
shit is hard to get over but sometimes the tragedy means it's over soulja..

---

Target: 1 (real disaster)
Text:
U.S. record hurricane drought. http://t.co/fE9hIVfMxq

---



### Split data into training and validation datasets



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of data for validation split
                                                                            random_state=42)

In [14]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [15]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

Tokenization, Embedding

Word2vec embedding:
http://jalammar.github.io/illustrated-word2vec/

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary/dictionary. If none, there's no cap on the size of the vocabulary. If set to an integer, every word that doesn't fit into the dictionary is tokenized as <OOV>
                                    standardize="lower_and_strip_punctuation", # turns all uppercase to lowercase and removes dots, spaces etc
                                    split="whitespace", # text is split into tokens with whitespace
                                    ngrams=None, # whether to arrange text into n-word groups
                                    output_mode="int", # assigns an int token to every word
                                    output_sequence_length=None, # converts every string into a sequence of the same length - the length of the longest sequence - and adds 0's at the end if words are absent
                                    pad_to_max_tokens=False)

In [22]:
len(train_sentences[0].split())

7

In [23]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [24]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in the vocabulary
max_length =15 # how many words in a tweet does the model see

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length) # the vectorizer truncates or pads the tweets to this length

In [25]:
# fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [26]:
# Create a sample sentence and vectorize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [30]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:\n{text_vectorizer([random_sentence])}")

Original text:
@Calum5SOS I need to stop doing this to myself???? @s_casualty        

Vectorized version:
[[4041    8  162    5  240  487   19    5  736 8765    0    0    0    0
     0]]
